In [31]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid
import pprint
import argparse

from graph import DataExtractor

In [41]:
import os

folder_path = './data/input_data/'  # 대상 폴더 경로

# 폴더 내의 모든 파일명 불러오기
file_names = [int(f[6:9]) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

print(file_names)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143]


In [38]:
pd.read_csv("./output/250110155353-Temp_CSV.csv")

,Paper Number,Stoichiometry information,Commercial NCM,Lithium source,Synthesis method,Crystallization method,Crystallization temperature,Crystallization time,Doping,Coating,...,Additive,Electrode thickness,Only Cathode Electrode diameter,Loading density (mass loading of NCM),Additional treatment for electrode,Explanation of SEM results,Explanation of TEM results,"Capacity at all C-rate, mAh/g (with electrode state)",Voltage range,Temperature
0,56,NCM-811,Yes,LiOH·H2O,co-precipitation,Sintering,750°C,15 hr,Mg2+ doping,Li3PO4 coating,...,NaN,NaN,NaN,NaN,NaN,Figure 9A; the pristine NCM electrode is damag...,NaN,"[{'203.5 mAh/g': '@0.1C, M-NCM', '180.0 mAh/g'...",3.0–4.3 V,NaN
1,139,Li[Ni0.73Co0.12Mn0.15]O2,No,LiOH,co-precipitation,NaN,750°C,20 hr,NaN,Al2O3 coating,...,NaN,NaN,NaN,2.5 mg/1.5 cm^2,NaN,"Fig. 2a, b, c; the pristine Li[Ni0.73Co0.12Mn0...",Fig. 3; the pristine material shows a very smo...,"[{'214 mAh/g': '@0.1C, FD-Al2O3-coated', '192 ...",3.0–4.3 V,Room temperature and 55°C


In [7]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [8]:
# 4개의 질문 입력
questions = [
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "CAM (Cathode Active Material)": {
                "Stoichiometry information": "NCM-622",
                "Commercial NCM": "No",
                "Lithium source": "LiOH",
                "Synthesis method": "co-precipitation",
                "Crystallization method": "Hydrothermal",
                "Crystallization temperature": "100°C",
                "Crystallization time": "12 hr",
                "Doping": "Zr4+ doping",
                "Coating": "ZrO2 coating",
                "Additional treatment": "None"
                }
        }
    ]
    """,    
    
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "Electrode (only for coin-cell (half-cell))": {
                "Active material : Conductive additive : Binder ratio": "90 : 5 : 5",
                "Electrolyte": "LiPF6 (EC, EMC, DEC mixture in a 1:1:1 volume ratio)",
                "Additive": "FEC 10% addition",
                "Electrode thickness": "100 µm",
                "Only Cathode Electrode diameter": "14π",
                "Loading density (mass loading of NCM)": "0.005 g/cm^2",
                "Additional treatment for electrode": "None"
                },
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {        
            "Morphological results": {
                "Explanation of SEM results": "Fig. 2a, b; the NCM-622 seems to have more or less a spherical morphology with a diameter of 3–5 µm, composed of densely packed primary particles",
                "Explanation of TEM results": "None"
                },    
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {            
            "Cathode Performance": {
                "Capacity at all C-rate, mAh/g (with electrode state)": [{
                "214.5 mAh/g": "@0.1C, ZrO2-coated",
                "200.8 mAh/g": "@0.5C, ZrO2-coated"
                }],
                "Voltage range": "2.8–4.3 V",
                "Temperature": "Room temperature and 55°C"
                }
        }
    ]
    """
]


In [16]:
total_outputs = {}
for file_num in [56, 139]:
    print(f"#####   {file_num}번째 논문    #####")
    voltai_graph = DataExtractor(
        # folder_path, 
        file_number=file_num
    ).graph

    # config 설정(재귀 최대 횟수, thread_id)
    config = RunnableConfig(
        recursion_limit=20, 
        configurable={"thread_id": random_uuid()}
    )

    # 4개의 질문에 대해 그래프 실행 및 출력
    results = []
    for i, question in enumerate(questions):
        # inputs = GraphState(question=question)
        print(f"{i+1}번째 질문")
        # print(question)
        result = voltai_graph.invoke(
            input={"question":question},
            config=config,
        )
        results.append(result)
    
    total_outputs[file_num] = results

#####   56번째 논문    #####
1번째 질문
==== [RELEVANCE CHECK] ====
yes
2번째 질문
==== [RELEVANCE CHECK] ====
no
==== [RELEVANCE CHECK] ====
yes
3번째 질문
==== [RELEVANCE CHECK] ====
yes
4번째 질문
==== [RELEVANCE CHECK] ====
yes
#####   139번째 논문    #####
1번째 질문
==== [RELEVANCE CHECK] ====
yes
2번째 질문
==== [RELEVANCE CHECK] ====
yes
3번째 질문
==== [RELEVANCE CHECK] ====
yes
4번째 질문
==== [RELEVANCE CHECK] ====
yes


In [17]:
total_outputs[56]

[{'question': 'Below are instructions for filling out items by referring to the examples.\n    The values shown to the right of the colon (“:”) are examples;\n    please delete them after reviewing and rewrite them with the values found in the PDF.\n    If any item is not mentioned in the PDF, do not remove it—write “None.”\n    [\n        {\n            "CAM (Cathode Active Material)": {\n                "Stoichiometry information": "NCM-622",\n                "Commercial NCM": "No",\n                "Lithium source": "LiOH",\n                "Synthesis method": "co-precipitation",\n                "Crystallization method": "Hydrothermal",\n                "Crystallization temperature": "100°C",\n                "Crystallization time": "12 hr",\n                "Doping": "Zr4+ doping",\n                "Coating": "ZrO2 coating",\n                "Additional treatment": "None"\n                }\n        }\n    ]\n    ',
  'context': 'prominently reduced. From Figure 3C, the O 1 s spec

In [25]:
total_outputs[56][0]["answer"]

[{'CAM (Cathode Active Material)': {'Stoichiometry information': 'LiNi0.8Co0.1Mn0.1O2',
   'Commercial NCM': 'Yes',
   'Lithium source': 'LiOH',
   'Synthesis method': 'co-precipitation',
   'Crystallization method': 'Sintering',
   'Crystallization temperature': '750°C',
   'Crystallization time': '15 hr',
   'Doping': 'Mg doping',
   'Coating': 'Li3PO4 coating',
   'Additional treatment': 'None'}}]

In [ ]:
import os
import time
import pandas as pd

def save_csv2output_folder(output_folder: str, df: pd.DataFrame, filename: str):
    # output 폴더 경로
    output_folder = output_folder
    
    # output 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 파일 이름 설정
    timestamp = time.strftime("%y%m%d%H%M%S")
    filename = timestamp + "-" + filename
    
    # CSV 파일 저장 경로
    output_path = os.path.join(output_folder, filename)
    
    # DataFrame을 CSV로 저장
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"{filename} 파일이 {output_path}에 저장되었습니다.")


def outputs2csv(total_outputs:dict):
    answers_list = []
    for file_num in list(total_outputs.keys()):
        outputs = total_outputs[file_num]
        answers = outputs[0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[2]["answer"][0]["Morphological results"] | outputs[3]["answer"][0]["Cathode Performance"]
        answers["Paper Number"] = file_num
        answers_list.append(answers)

    answers_csv = pd.DataFrame(answers_list)                               
    columns = ["Paper Number"] + [col for col in answers_csv.columns if col != "Paper Number"]
    answers_csv = answers_csv[columns]
    
    save_csv2output_folder(output_folder="./output/", df=answers_csv, filename="Temp_CSV")
    
def outputs2pprint(total_outputs:dict):
    for file_num in list(total_outputs.keys()):
        print(f"##  {file_num}번째 논문 결과")
        outputs = total_outputs[file_num]
        answers = outputs[0]["answer"][0] | outputs[1]["answer"][0] | outputs[2]["answer"][0] | outputs[3]["answer"][0]
        pprint.pprint(answers)

In [29]:
answers56 = outputs[56][0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[56][1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[56][2]["answer"][0]["Morphological results"] | outputs[56][3]["answer"][0]["Cathode Performance"]
answers139 = outputs[139][0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[139][1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[139][2]["answer"][0]["Morphological results"] | outputs[139][3]["answer"][0]["Cathode Performance"]

In [30]:
pd.DataFrame([answers56, answers139])

,Stoichiometry information,Commercial NCM,Lithium source,Synthesis method,Crystallization method,Crystallization temperature,Crystallization time,Doping,Coating,Additional treatment,...,Additive,Electrode thickness,Only Cathode Electrode diameter,Loading density (mass loading of NCM),Additional treatment for electrode,Explanation of SEM results,Explanation of TEM results,"Capacity at all C-rate, mAh/g (with electrode state)",Voltage range,Temperature
0,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH,co-precipitation,Sintering,750°C,15 hr,Mg doping,Li3PO4 coating,None,...,None,None,None,None,None,"Fig. 2A, B; the pristine NCM and M-NCM exhibit...",None,"[{'203.5 mAh/g': '@0.1C, M-NCM', '180.0 mAh/g'...",3.0–4.3 V,None
1,Li[Ni0.73Co0.12Mn0.15]O2,No,LiOH,co-precipitation,Not explicitly mentioned,750°C,20 hr,None,Al2O3 coating,None,...,None,None,None,None,None,"Fig. 2a, b, c; the pristine Li[Ni0.73Co0.12Mn0...","Fig. 3a, b, c; The pristine material shows a s...","[{'214 mAh/g': '@0.1C, FD-Al2O3-coated', '226 ...",3.0–4.3 V,Room temperature and 55°C


In [13]:
seperated_outputs = outputs[0]["answer"][0] | outputs[1]["answer"][0] | outputs[2]["answer"][0] | outputs[3]["answer"][0]
pprint.pprint(seperated_outputs)

TypeError: list indices must be integers or slices, not str

In [11]:
outputs_dict = outputs[0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[2]["answer"][0]["Morphological results"] | outputs[3]["answer"][0]["Cathode Performance"]
pprint.pprint(outputs_dict)

{'Active material : Conductive additive : Binder ratio': 'None',
 'Additional treatment': 'calcination',
 'Additional treatment for electrode': 'Dried in a vacuum oven at 80°C for 12 '
                                       'h',
 'Additive': '2 wt% VC',
 'Capacity at all C-rate, mAh/g (with electrode state)': [{'113.0 mAh/g': '@0.5C, '
                                                                          'NCM622-600, '
                                                                          '100th '
                                                                          'cycle',
                                                           '126.1 mAh/g': '@5C, '
                                                                          'NCM622-700',
                                                           '129.3 mAh/g': '@0.5C, '
                                                                          'NCM622-800, '
                                                                

In [10]:
pd.DataFrame(outputs_dict)

,Stoichiometry information,Commercial NCM,Lithium source,Synthesis method,Crystallization method,Crystallization temperature,Crystallization time,Doping,Coating,Additional treatment,...,Additive,Electrode thickness,Only Cathode Electrode diameter,Loading density (mass loading of NCM),Additional treatment for electrode,Explanation of SEM results,Explanation of TEM results,"Capacity at all C-rate, mAh/g (with electrode state)",Voltage range,Temperature
0,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH/H2O,Co-precipitation,Calcination and sintering,750°C,15 hr,Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,Fig. 9A; the cycled electrode of the pristine ...,None,"{'203.5 mAh/g': '@0.1C, M-NCM', '180.0 mAh/g':...",3.0–4.3 V,None
